In [0]:
#6주차

data = sc.parallelize([('Amber',22),('Alfred',23),('Skye',4),('Albert',12),('Amber',9)])
print(data.collect())
print(data.take(2))

[('Amber', 22), ('Alfred', 23), ('Skye', 4), ('Albert', 12), ('Amber', 9)]
[('Amber', 22), ('Alfred', 23)]

In [0]:
#1,map : 형태를 변화하여 저장
x = sc.parallelize(['b','a','c'])
y = x.map(lambda x : (x,1))
print(y.collect())

[('b', 1), ('a', 1), ('c', 1)]

In [0]:
#2,filter : 조건에 맞는 거 걸러냄
x = sc.parallelize([1,2,3])
y = x.filter(lambda x : x % 2 ==1)
print(y.collect())

[1, 3]

In [0]:
#3,flatMap : 나란히 이어붙힘
x = sc.parallelize([1,2,3])
y = x.flatMap(lambda x : (x,x*100,42))
print(y.collect())

[1, 100, 42, 2, 200, 42, 3, 300, 42]

In [0]:
#4,groupBy : 키를 직접 생성하여 저장
x = sc.parallelize(['John','Amber','Kain'])
y = x.groupBy(lambda w : w[0])
print(list((k, list(v)) for (k,v) in y.collect()))

[('J', ['John']), ('K', ['Kain']), ('A', ['Amber'])]

In [0]:
#5,groupByKey : 이미 키가 있는 것에 대하여 리스트로 합침
x = sc.parallelize([('J','John'),('A','Amber'),('J','James')])
y = x.groupByKey()
print(list((k, list(v)) for (k,v) in y.collect()))

[('J', ['John', 'James']), ('A', ['Amber'])]

In [0]:
#6, resudeByKey : 같은 키에 대하여 합쳐서 보통 개수 셀 때, 어떻게 합쳐 줄 것인지는 함수 형태를 넣어서 지정
words = ['one','two','three','four','one','two']
x = sc.parallelize(words).map(lambda x : (x,1))
print(x.collect())
y = x.reduceByKey(lambda x,y : x+y)
print(y.collect())
print("==================")
y = x.groupByKey()
print(list((k, list(v)) for (k,v) in y.collect()))
y = y.map(lambda x : (x[0],sum(x[1])))
print(y.collect())

[('one', 1), ('two', 1), ('three', 1), ('four', 1), ('one', 1), ('two', 1)]
[('two', 2), ('three', 1), ('four', 1), ('one', 2)]
==================
[('two', [1, 1]), ('three', [1]), ('four', [1]), ('one', [1, 1])]
[('two', 2), ('three', 1), ('four', 1), ('one', 2)]

In [0]:
#파티션이 어떻게 나눠 져 있는지 : glom
import numpy as np

data = sc.parallelize(np.arange(20))
print(data.glom().collect())
#직접 지정
data = sc.parallelize(np.arange(20),4)
print(data.glom().collect())
print(data.collect())

[[0, 1], [2, 3], [4, 5], [6, 7, 8, 9], [10, 11], [12, 13], [14, 15], [16, 17, 18, 19]]
[[0, 1, 2, 3, 4], [5, 6, 7, 8, 9], [10, 11, 12, 13, 14], [15, 16, 17, 18, 19]]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

In [0]:
x = sc.parallelize(np.arange(10),2)
def f(iterator):
  yield sum(iterator)
y = x.mapPartitions(f) 
print(x.glom().collect())
print(y.glom().collect())
print(y.collect())

[[0, 1, 2, 3, 4], [5, 6, 7, 8, 9]]
[[10], [35]]
[10, 35]

In [0]:
def f(partitionIndex, iterator):
  yield (partitionIndex,sum(iterator))

y = x.mapPartitionsWithIndex(f)
 
print(x.glom().collect())
print(y.glom().collect())
print(y.collect())

[[0, 1, 2, 3, 4], [5, 6, 7, 8, 9]]
[[(0, 10)], [(1, 35)]]
[(0, 10), (1, 35)]